In [12]:
import numpy as np
import pandas as pd
# Drawdown Máximo
import yfinance as yf

In [13]:
df = yf.Ticker('NQ=F')
df = df.history(start='2010-01-01', end='2024-01-01')

In [14]:
def backtest_strategy(sma_1, sma_2):
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()

    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    df.dropna(inplace=True)

    # Calcular drawdown máximo
    df['Cumulative_Returns'] = (df['Strategy_Returns'] + 1).cumprod()
    df['Peak'] = df['Cumulative_Returns'].cummax()
    df['Drawdown'] = (df['Cumulative_Returns'] - df['Peak']) / df['Peak']
    max_drawdown = df['Drawdown'].min()
    
    return max_drawdown

### Optimización mediante Grid Search


In [15]:
best_params = None
best_drawdown = np.inf

In [16]:
for sma_1 in range(10, 100, 10):
    for sma_2 in range(100, 300, 50):
        drawdown = backtest_strategy(sma_1, sma_2)
        print(f'Probando SMA_1: {sma_1}, SMA_2: {sma_2} - Máximo Drawdown: {drawdown:.2%}')
        
        if abs(drawdown) < abs(best_drawdown):  # Compara la magnitud del drawdown
            best_drawdown = drawdown
            best_params = (sma_1, sma_2)

Probando SMA_1: 10, SMA_2: 100 - Máximo Drawdown: -66.29%
Probando SMA_1: 10, SMA_2: 150 - Máximo Drawdown: -54.39%
Probando SMA_1: 10, SMA_2: 200 - Máximo Drawdown: -46.01%
Probando SMA_1: 10, SMA_2: 250 - Máximo Drawdown: -39.38%
Probando SMA_1: 20, SMA_2: 100 - Máximo Drawdown: -45.21%
Probando SMA_1: 20, SMA_2: 150 - Máximo Drawdown: -42.76%
Probando SMA_1: 20, SMA_2: 200 - Máximo Drawdown: -43.90%
Probando SMA_1: 20, SMA_2: 250 - Máximo Drawdown: -44.37%
Probando SMA_1: 30, SMA_2: 100 - Máximo Drawdown: -43.40%
Probando SMA_1: 30, SMA_2: 150 - Máximo Drawdown: -39.59%
Probando SMA_1: 30, SMA_2: 200 - Máximo Drawdown: -32.03%
Probando SMA_1: 30, SMA_2: 250 - Máximo Drawdown: -36.24%
Probando SMA_1: 40, SMA_2: 100 - Máximo Drawdown: -42.12%
Probando SMA_1: 40, SMA_2: 150 - Máximo Drawdown: -31.81%
Probando SMA_1: 40, SMA_2: 200 - Máximo Drawdown: -28.51%
Probando SMA_1: 40, SMA_2: 250 - Máximo Drawdown: -20.40%
Probando SMA_1: 50, SMA_2: 100 - Máximo Drawdown: -23.65%
Probando SMA_1

In [17]:
print(f'Los mejores parámetros son: {best_params} con un Máximo Drawdown de {best_drawdown:.2%}')

Los mejores parámetros son: (50, 250) con un Máximo Drawdown de -1.69%
